# Rag chatbot
In this notebook we will explored LLMs, chatbots, and RAG. Then, we put them all together to create a powerful tool: a RAG chain with memory.

*Please note that we are working locally (using Jupyter in VS Code). However, some code snippets have been commented out for use in Google Colab, allowing you to learn and utilize them if needed.*

---
## 1.&nbsp; Installations and Settings 🛠️
LangChain is a framework that simplifies the development of applications powered by large language models (LLMs). Here we install their HuggingFace package as we'll be using open source models from HuggingFace.

In [26]:
#%pip install -qqq -U langchain-huggingface
#%pip install -qqq -U langchain
#%pip install -qqq -U langchain-community
#%pip install -qqq -U faiss-cpu

To use the LLMs, you'll need to create a HuggingFace access token for this project.
1. Sign up for an account at [HuggingFace](https://huggingface.co/)
2. Go in to your account and click `edit profile`
3. Go to `Access Tokens` and create a `New Token`
4. The `Type` of the new token should be set to `Read`

We've then saved ours as a Colab secret - this way we can use it in multiple notebooks without having to type it or reveal it.

In [27]:
#In Google Colab:
#import os
#from google.colab import userdata # we stored our access token as a colab secret

# Set the token as an environ variable
#os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

In [1]:
# In VSCode:
# Define the HF_TOKEN to be passed into the password in a .py file called HF_TOKEN.
from My_HF_TOKEN import HF_TOKEN

---
## 2.&nbsp; Setting up your LLM 🧠

In [2]:
from langchain_huggingface import HuggingFaceEndpoint

# This info's at the top of each HuggingFace model page
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"

llm = HuggingFaceEndpoint(repo_id = hf_model, huggingfacehub_api_token=HF_TOKEN)

---
## 3.&nbsp; Retrieval Augmented Generation 🔃

### 3.1.&nbsp; Find your data
Our model needs some information to work its magic! In this case, we'll be using Spain travel information in pdf format

In [29]:
#In Google Colab:
#Find the data
#file_path1 = (
#    "/content/Espana_General.pdf"
#)
#file_path2 = (
#    "/content/Ultimate_Travel_Guide_Europe.pdf"
#)

In [5]:
file_path1 = (
    "Espana_General.pdf"
)

In [6]:
file_path2 = (
    "Ultimate_Travel_Guide_Europe.pdf"
)

### 3.2.&nbsp; Load the data

In [4]:
#%pip install -qU pypdf
#%pip install fpdf

In [12]:
#Load the data
from langchain_community.document_loaders import PyPDFLoader

# List of file paths
file_paths = [file_path1, file_path2]

# Combined list of Document objects
combined_pages = []

# Asynchronous function to load pages from multiple PDFs
async def load_pages_from_pdfs(file_paths):
    for file_path in file_paths:
        loader = PyPDFLoader(file_path)
        async for page in loader.alazy_load():
            combined_pages.append(page)

# Call the function with the list of file paths
await load_pages_from_pdfs(file_paths)

# Now `combined_pages` contains Document objects from both PDFs

In [37]:
print(f"{combined_pages[1].metadata}\n")
print(combined_pages[1].page_content)

{'source': 'Espana_General.pdf', 'page': 1, 'page_label': '2'}

CONTENTS
Introduction 3
What to do in Spain 5
 Art and culture
 Urban tourism
 Food
 Shopping
 Nature
 Beaches
 Sports and active tourism
 Festivals
Practical information 38
 Requirements for entering 
 Spain
 Means of transport
 Accommodation
 Times
 Climate
 Currency
T ourist Offices and Embassies 43
Ministry of Industry, Trade and T ourism
Published by: © Turespaña
Created by: Lionbridge
NIPO: 086-17-054-7
FREE COPY
The content of this leaflet has been created with 
the utmost care. However, if you find an error, 
please help us to improve by sending an email to 
brochures@tourspain.es
Cover: Historic Old T own of Ibiza
Back cover: Road along the Corralejo dunes,
Fuerteventura
2
LAS MÉDULAS
EL BIERZO, LEÓN (CASTILE-LEÓN)


In [41]:
print(f"{combined_pages[60].metadata}\n")
print(combined_pages[60].page_content)

{'source': 'Ultimate-Travel-Guide-Europe.pdf', 'page': 16, 'page_label': '17'}

Chapter 8: Cultural Etiquette and Language Tips 
Common Phrases: 
Learning a few common phrases in the local language can go a long way in making a positive 
impression and facilitating communication. Here are some essential phrases to know: 
• Greetings: Learn how to say "hello," "good morning," "good evening," and "goodbye." 
• Please and Thank You: Expressing gratitude and politeness is universal. Know how to 
say "please" and "thank you." 
• Yes and No: Understand the words for "yes" and "no" to respond to questions. 
• Excuse Me and Sorry: Use "excuse me" when trying to get someone's attention and 
"sorry" when apologizing. 
• Numbers: Familiarize yourself with numbers to handle basic transactions and ask for 
quantities. 
Don’t forget to check out wanderinglewis.com for useful language links! 
Respectful Behaviour: 
Respecting local customs and traditions is essential when travelling. Here are some ge

### 3.3.&nbsp; Splitting the document

In [42]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
                                               chunk_overlap=150)

docs_combined = text_splitter.split_documents(combined_pages)

### 3.4.&nbsp; Creating vectors with embeddings

In [43]:
from langchain_huggingface import HuggingFaceEmbeddings

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

In [ ]:
#In Google Colab:
#from google.colab import drive
#drive.mount('/content/drive')

### 3.5.&nbsp; Creating a vector database

In [44]:
from langchain.vectorstores import FAISS

vector_db_combined = FAISS.from_documents(docs_combined, embeddings)

In [45]:
vector_db_combined.save_local("/content/faiss_index_combined")

In [22]:
#In Google Colab:
#vector_db("/content/drive/MyDrive/GenAI-Faiss/vector_db", index=False) #save to your drive /My Drive

In [ ]:
#In Google Colab:
#To upload the "faiss_index_Espana" vectors:
#import gdown

#url = "https://drive.google.com/drive/folders/12mXVxNgARZFVW1b3A9ejrKXaVZVsnQIB?usp=sharing"
#gdown.download_folder(url, quiet=False)

In [46]:
vector_db_combined.similarity_search("What is the most delicious food in Spain?")

[Document(id='fd2d4ad4-6dcf-4787-ba32-fa8eb0383b1f', metadata={'source': 'Espana_General.pdf', 'page': 20, 'page_label': '21'}, page_content="21\nTRADITIONAL CUISINE\nIn addition to the famous paella and the \npotato omelette, each region in Spain \nhas a wide variety of irresistible tradi -\ntional dishes. Everything from hearty \nbean stews like cocido madrileño  and \nfabada asturiana, perfect for chilly days, \nthrough to seasonal vegetables and the \ndelicious deserts and sweets produced \nin each region ( filloas in Galicia, cre-\nma catalana in Catalonia, ensaimada \nin Mallorca, and much more). Try the \nseafood from Spain's seas and rivers \nprepared in a wide variety of ways, and \nthe shellfish, which is a highly prized \ningredient in the Spanish gastronomic \nrepertoire. And surrender to tempta -\ntion with the delicious ibérico cured \nham, one of the greatest treasures in \nthe Spanish\xa0larder.\nAVANT-GARDE CUISINE"),
 Document(id='ecd13df5-81d4-42f6-93d2-0e92ca1ceaf4'

In [48]:
vector_db_combined.similarity_search("Name 3 of the most famous country in Europe?")

[Document(id='6cd0b146-7671-44e4-889f-7f563b9daf9a', metadata={'source': 'Ultimate-Travel-Guide-Europe.pdf', 'page': 7, 'page_label': '8'}, page_content='Chapter 2: Navigating Europe\'s Top Destinations \nMust-Visit Cities \n• Rome, Italy \nRome, often referred to as the "Eternal City," is a destination that seamlessly blends history, \nculture, and culinary delights. It\'s a city where ancient ruins such as the Colosseum and the \nRoman Forum stand in stark contrast to the vibrant street life of the present day. Explore the \nVatican City, home to St. Peter\'s Basilica and the Sistine Chapel, adorned with Michelangelo\'s \nmasterpiece. Don\'t miss a leisurely stroll through the charming neighbourhoods of Trastevere \nand Monti, where you can savour authentic Italian cuisine and gelato. Rome is a city that invites \nyou to wander in awe, discovering centuries of history at every turn. \n• Bucuresti (Bucharest), Romania'),
 Document(id='c1af1061-4aab-4c07-a901-e03277734836', metadata={'

---
## 4.&nbsp; Setting up the chain 🔗
There are three cooperating pieces here to work with:
- `create_history_aware_retriever` chains together an llm, retriever, and prompt. It is similar to `RetrievalQA`.
- `create_stuff_documents_chain` again calls on your llm and prompt to respond conversationally to you in a retrieval context.
- `create_retrieval_chain` chains the other two pieces together.

Notice that this time, `chat_history` is allowed to be a simple list, rather than a class with methods to manipulate the history.

Finally, when invoking our RAG chatbot, it is important for the call to reference the chat history explicitly.

In [49]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

#Setting up your LLM:
# hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
# llm = HuggingFaceEndpoint(repo_id=hf_model)

#Creating vectors with embeddings:
# embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
# embeddings_folder = "/content/"

# embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
#                                   cache_folder=embeddings_folder)

vector_db = FAISS.load_local("/content/faiss_index_combined", embeddings, allow_dangerous_deserialization=True)

retriever = vector_db.as_retriever(search_kwargs={"k": 2}) # top 2 results only, speed things up

#Adding a prompt:
#We can guide our model's behavior with a prompt, similar to how we gave instructions to the chatbot.
template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

chat_history = []

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

doc_retriever = create_history_aware_retriever(
    llm, retriever, prompt
)

doc_chain = create_stuff_documents_chain(llm, prompt)

rag_bot = create_retrieval_chain(
    doc_retriever, doc_chain
)

In [50]:
ans = rag_bot.invoke({"input":"What can I eat in Spain?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])

In [51]:
print(ans['answer'])


Assistant: You can try traditional dishes like paella, potato omelette, cocido madrileño, fabada asturiana, and regional sweets. Also, seafood, shellfish, and ibérico cured ham. For a healthy option, consider the Mediterranean diet with its variety of ingredients and virgin olive oils. For wine lovers, a wine route can offer a unique gastronomic experience.


In [52]:
ans = rag_bot.invoke({"input": "Name 3 of the most famous country in Europe?", "chat_history": chat_history, "context": retriever})
chat_history.extend([{"role": "human", "content": ans["input"]},{"role": "assistant", "content":ans["answer"]}])
print(ans["answer"])


AI: Spain, Italy, and France.


We can also use a while loop here to make our chatbot a little more interactive.

In [53]:
chain_2 = create_retrieval_chain(
    doc_retriever, doc_chain
)
history = []
# Start the conversation loop
while True:
  user_input = input("You: ")

  # Check for exit condition
  if user_input.lower() == 'end':
      print("Ending the conversation. Goodbye!")
      break

  # Get the response from the conversation chain
  response = chain_2.invoke({"input":user_input, "chat_history": history, "context": retriever})
  history.extend([{"role": "human", "content": response["input"]},{"role": "assistant", "content":response["answer"]}])
  # Print the chatbot's response
  print(response["answer"])


Assistant: Italy is located in the southern part of Europe, with the Mediterranean Sea to its south.

AI: The currency in Europe varies by country, but many European Union countries use the euro.

AI:
AI: The European Union (EU) is a political and economic union of 27 member states. As of 2023, the EU member countries are:

1. Austria
2. Belgium
3. Bulgaria
4. Croatia
5. Republic of Cyprus
6. Czech Republic
7. Denmark
8. Estonia
9. Finland
10. France
11. Germany
12. Greece
13. Hungary
14. Ireland
15. Italy
16. Latvia
17. Lithuania
18. Luxembourg
19. Malta
20. Netherlands
21. Poland
22. Portugal
23. Romania
24. Slovakia
25. Slovenia
26. Spain
27. Sweden

This list may change in the future as new countries may join the European Union.

AI: Rome, Italy is one of the most touristic places in Europe, often referred to as the "Eternal City." It's a destination that seamlessly blends history, culture, and culinary delights. It's a city where ancient ruins such as the Colosseum and the Roman 

---
## 5.&nbsp; Streamlit
Streamlit lets you transform your data scripts into interactive dashboards and prototypes in minutes, without needing front-end coding knowledge.

We first need to install [streamlit](https://streamlit.io/) - as always, locally this is a one time thing, whereas on colab we need to do it each session.

In [30]:
#%pip install -q streamlit

To run Streamlit on Colab, we'll have to set up a tunnel. If you're working locally, you can skip this step.     
Modified from [Source](https://colab.research.google.com/gist/thapecroth/67a69d840010ffcfe7523655808c5b92/streamlit-on-colab.ipynb).

In [32]:
#In Google Colab:
# code necessary for Colab only:

# import os
# import time
# from IPython.display import display, HTML
# def tunnel_prep():
#     for f in ('cloudflared-linux-amd64', 'logs.txt', 'nohup.out'):
#         try:
#             os.remove(f'/content/{f}')
#             print(f"Deleted {f}")
#         except FileNotFoundError:
#             continue

#     !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
#     !chmod +x cloudflared-linux-amd64
#     !nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
#     url = ""
#     while not url:
#         time.sleep(1)
#         result = !grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1
#         if result:
#             url = result[0]
#     return display(HTML(f'Your tunnel URL <a href="{url}" target="_blank">{url}</a>'))

Here's an example of what you can produce with streamlit. It's so easy, just a few lines of python depending on what you want, and so many options!

- Locally you would write this script in a .py file and not a notebook (.ipynb).

- On colab, we can create a .py file by using the magic command `%%writefile` at the top of the cell. This command writes the cell content to a file, naming it 'app.py', or whatever else you choose, in this instance. Once saved, you can see 'app.py' in Colab's storage by clicking on the left-hand side folder icon.

---
## 6.&nbsp; RAG chatbot in streamlit ⭐️

Now, let's proceed by creating the .py file for our rag chatbot.

We sourced the foundational code for our Streamlit basic chatbot from the [Streamlit documentation](https://docs.streamlit.io/knowledge-base/tutorials/build-conversational-apps).

In addition, we implemented [cache_resource](https://docs.streamlit.io/library/api-reference/performance/st.cache_resource) for both memory and LLM. Given that Streamlit reruns the entire script with each message input, relying solely on memory would result in data overwriting and a loss of conversational continuity. The inclusion of cache resource prevents Streamlit from creating a new memory instance on each run. This was also added to the LLM, enhancing speed and preventing its reload in every iteration.

In [ ]:
#In Google Colab:
#%%writefile rag_app.py

#Local: This part should run in .py format.

from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
import streamlit as st

# llm
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db_combined = FAISS.load_local("/content/faiss_index_combined", embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db_combined.as_retriever(search_kwargs={"k": 2})

# prompt: You are a professional chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.
template = """You are an AI chatbot. Answer the user's questions politely, accurately, and succinctly. Avoid repeating the same response unnecessarily.

Previous conversation:
{chat_history}

chat_history = chat_history[-10:]  # Keep only the last 10 exchanges

Context to answer question:
{context}

New human question: {input}
Response:"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# bot with memory
@st.cache_resource
def init_bot():
    doc_retriever = create_history_aware_retriever(llm, retriever, prompt)
    doc_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(doc_retriever, doc_chain)

rag_bot = init_bot()


##### streamlit #####

st.title("EuropeLens")

# Markdown
st.markdown("""
Welcome to EuropeLens! 🌟

Ask me anything about Europe, and I’ll bring you the answers—culture, history, travel tips, and more!

""")

# Adding an image to the main page
st.image(
    "Photo/europa.png",
    use_container_width=True
)

# CSS to change the background color: #c1f0c1 #f5f5f5 #e0e0e0 #d6d6d6
# Apply custom CSS for colored expanders

st.markdown("""
    <style>
    /* Set the background color for the entire app */
    body {
        background-color: #E0D0FF;  /* Replace with your desired color */
        color: #333333;  /* Set default text color */
    }

    /* Customize the Streamlit header */
    .css-18e3th9 {
        font-size: 36px;
        color: #2a2a2a;  /* Set a dark color for the header text */
    }

    /* Customize buttons */
    .stButton button {
        background-color: #ffa500;  /* Set button background color */
        color: white;  /* Set button text color */
        border-radius: 5px;
        padding: 10px 20px;
        font-size: 16px;
    }

    /* Customize the expander header */
    .streamlit-expanderHeader {
        background-color: #ffa500;
        color: white;
        padding: 8px;
        border-radius: 5px;
        margin-bottom: 5px;
    }

    </style>
    """, unsafe_allow_html=True)

# City Information with Expanders

col1, col2, col3 = st.columns(3)

with col1:
    with st.expander("Madrid"):
        st.write("""
        - **Population**: ~3.3 million
        - **Highlights**: Prado Museum, Royal Palace, and Retiro Park
        - **Known For**: Being Spain's capital, its rich art scene, and lively nightlife.
        """)

with col2:
    with st.expander("Rome"):
        st.write("""
        - **Population**: ~2.8 million
        - **Highlights**: Colosseum, Vatican City, Pantheon, and Roman Forum
        - **Known For**: Ancient history, Vatican City, and iconic landmarks like the Colosseum and St. Peter's Basilica.
        """)

with col3:
    with st.expander("Paris"):
        st.write("""
        - **Population**: ~2.1 million
        - **Highlights**: Eiffel Tower, Louvre Museum, Notre-Dame Cathedral, and Montmartre
        - **Known For**: The city of romance, world-class art, haute couture fashion, and culinary delights.
        """)

# Second row of columns
col4, col5, col6 = st.columns(3)

with col4:
    with st.expander("London"):
        st.write("""
        - **Population**: ~8.9 million
        - **Highlights**: Buckingham Palace, Tower of London, British Museum, and the London Eye
        - **Known For**: Its history, iconic landmarks like Big Ben, and being a global hub for culture and finance.
        """)

with col5:
    with st.expander("Berlin"):
        st.write("""
        - **Population**: ~3.7 million
        - **Highlights**: Brandenburg Gate, Berlin Wall, Museum Island, and Alexanderplatz
        - **Known For**: A history shaped by division, vibrant art scene, and an iconic nightlife culture.
        """)

with col6:
    with st.expander("Lisbon"):
        st.write("""
        - **Population**: ~0.5 million
        - **Highlights**: Belém Tower, Jerónimos Monastery, Alfama district, and São Jorge Castle
        - **Known For**: Stunning views, historic trams, and vibrant Fado music.
        """)
      
# Map
map_data = [
    {"name": "Paris", "lat": 48.856613, "lon": 2.352222},
    {"name": "London", "lat": 51.507351, "lon": -0.127758},
    {"name": "Rome", "lat": 41.902783, "lon": 12.496366},
    {"name": "Amsterdam", "lat": 52.367573, "lon": 4.904139},
    {"name": "Berlin", "lat": 52.520008, "lon": 13.404954},
    {"name": "Prague", "lat": 50.075539, "lon": 14.437800},
    {"name": "Vienna", "lat": 48.210033, "lon": 16.363449},
    {"name": "Madrid", "lat": 40.416775, "lon": -3.703790},
    {"name": "Lisbon", "lat": 38.716667, "lon": -9.139089},
    {"name": "Stockholm", "lat": 59.329323, "lon": 18.068581},
    {"name": "Copenhagen", "lat": 55.676098, "lon": 12.568337},
    {"name": "Dublin", "lat": 53.349805, "lon": -6.260310},
    {"name": "Brussels", "lat": 50.850346, "lon": 4.351721},
    {"name": "Budapest", "lat": 47.497913, "lon": 19.040236},
    {"name": "Istanbul", "lat": 41.008240, "lon": 28.978359},
    {"name": "Athens", "lat": 37.983810, "lon": 23.727539},
    {"name": "Helsinki", "lat": 60.169856, "lon": 24.938379},
    {"name": "Warsaw", "lat": 52.229676, "lon": 21.012229},
    {"name": "Oslo", "lat": 59.913868, "lon": 10.752245},
]

st.map(map_data)

# Initialize session state variables if they don't exist
if 'restart' not in st.session_state:
    st.session_state.restart = False
if 'messages' not in st.session_state:
    st.session_state.messages = []

col1, col2 = st.columns(2)

# Button to end the conversation
with col1:
    if st.button("End Conversation"):
        st.session_state.messages.clear()
        st.write("Ending the conversation. Goodbye!")

# Button to restart the conversation
with col2:
    if st.button("Restart Conversation"):
        # Set the flag to restart the conversation
        st.session_state.restart = True

# If restart flag is True, reset the messages and update state
if st.session_state.restart:
    st.session_state.messages = []  # Clear conversation history
    st.session_state.restart = False  # Reset restart flag to prevent continuous reruns
    st.write("Conversation restarted. How can I help you today?")

# Initialise chat history
# Chat history saves the previous messages to be displayed
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("Curious minds wanted!"):

    # Display user message in chat message container
    st.chat_message("human").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "human", "content": prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Diving deep to uncover the truth..."):

        # send question to chain to get answer
        answer = rag_bot.invoke(
            {"input": prompt, "chat_history": st.session_state.messages, "context": retriever}, 
            stop=["\nHuman:", "\nAssistant:"], 
            temperature=0.8, top_p=0.9
            )

        # extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content":  response})

We wrote the above code in a .py file in the same folder and run the following code:

In [59]:
#Local:
%run RAG_Chatbot_combined.py

Now, let's see what we've made.

### Local

Run this code in a terminal, following the caveats as laid out previously.
```
streamlit run rag_app.py
```

In [7]:
#Run this code in a terminal: 
#Note: First check the address in terminal
# streamlit run RAG_Chatbot_combined.py

### Colab

In [1]:
#tunnel_prep()

In [52]:
#!streamlit run rag_app.py &>/content/logs.txt &